## Paso 2

In [1]:
# qsar_preprocessing.py
import pandas as pd
from sklearn.datasets import fetch_openml

# Cargar el dataset desde OpenML
qsar = fetch_openml(name="qsar-biodeg", version=1, as_frame=True)
df = qsar.frame

# Información básica
print("🔹 Dimensiones del dataset:", df.shape)
print("\n🔹 Columnas:")
print(df.columns.tolist())
print("\n🔹 Primeras filas:")
print(df.head())

# Revisar la variable objetivo
print("\n🔹 Valores únicos de la variable objetivo 'class':")
print(df['Class'].value_counts())


🔹 Dimensiones del dataset: (1055, 42)

🔹 Columnas:
['V1', 'V2', 'V3', 'V4', 'V5', 'V6', 'V7', 'V8', 'V9', 'V10', 'V11', 'V12', 'V13', 'V14', 'V15', 'V16', 'V17', 'V18', 'V19', 'V20', 'V21', 'V22', 'V23', 'V24', 'V25', 'V26', 'V27', 'V28', 'V29', 'V30', 'V31', 'V32', 'V33', 'V34', 'V35', 'V36', 'V37', 'V38', 'V39', 'V40', 'V41', 'Class']

🔹 Primeras filas:
      V1      V2  V3  V4  V5  V6  V7    V8  V9  V10  ...  V33  V34  V35  \
0  3.919  2.6909   0   0   0   0   0  31.4   2    0  ...    0    0    0   
1  4.170  2.1144   0   0   0   0   0  30.8   1    1  ...    0    0    0   
2  3.932  3.2512   0   0   0   0   0  26.7   2    4  ...    0    0    1   
3  3.000  2.7098   0   0   0   0   0  20.0   0    2  ...    0    0    1   
4  4.236  3.3944   0   0   0   0   0  29.4   2    4  ...    0    0    0   

     V36    V37  V38    V39  V40  V41  Class  
0  2.949  1.591    0  7.253    0    0      2  
1  3.315  1.967    0  7.257    0    0      2  
2  3.076  2.417    0  7.601    0    0      2  
3  

In [2]:
print(df['Class'].unique()[:10])

['2', '1']
Categories (2, object): ['1', '2']


In [3]:
# Asegurar tipo numérico y renombrar por claridad
df['Class'] = df['Class'].astype(int)

# Convertir a 0 y 1 (opcional, pero útil para modelos)
df['Class'] = df['Class'].map({2: 0, 1: 1})

print(df['Class'].value_counts())

Class
1    699
0    356
Name: count, dtype: int64


In [4]:
print("Valores únicos en la columna objetivo:")
print(df['Class'].unique())

print("\nCantidad de valores nulos en 'Class':")
print(df['Class'].isna().sum())


Valores únicos en la columna objetivo:
[0 1]

Cantidad de valores nulos en 'Class':
0


In [5]:
from sklearn.model_selection import train_test_split

X = df.drop('Class', axis=1)
y = df['Class']

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

print("\n🔹 Tamaños:")
print("Entrenamiento:", X_train.shape, " | Prueba:", X_test.shape)



🔹 Tamaños:
Entrenamiento: (844, 41)  | Prueba: (211, 41)


In [6]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

## Paso 3

In [7]:
import mlflow
import mlflow.sklearn  # Para modelos de Scikit-learn

In [8]:
# Nombre de tu experimento
experiment_name = "QSAR_Biodegradation_LogisticRegression"

# Crear o seleccionar el experimento
mlflow.set_experiment(experiment_name)

<Experiment: artifact_location='file:///C:/Users/Christian/Documents/mlflow%202/mlruns/468532580829688563', creation_time=1760587998401, experiment_id='468532580829688563', last_update_time=1760587998401, lifecycle_stage='active', name='QSAR_Biodegradation_LogisticRegression', tags={'mlflow.experimentKind': 'custom_model_development'}>

In [9]:
with mlflow.start_run():
    # Aquí dentro iría el entrenamiento de tu modelo
    mlflow.log_param("modelo", "Regresion Logistica")
    mlflow.log_param("test_size", 0.2)
    
    # Ejemplo: después de entrenar el modelo
    # mlflow.log_metric("accuracy", accuracy)
    
    print("Run iniciado. Ahora se pueden registrar parámetros y métricas.")

Run iniciado. Ahora se pueden registrar parámetros y métricas.


## Paso 4

In [10]:
# 🔹 Importar librerías necesarias
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, ConfusionMatrixDisplay
import matplotlib.pyplot as plt
import mlflow
import mlflow.sklearn

# 🔹 Configurar experimento
mlflow.set_experiment("QSAR_Biodegradation_LogisticRegression")

# 🔹 Iniciar un run en MLflow
with mlflow.start_run(run_name="LogisticRegression_Manual"):

    # Registrar parámetros que vas a probar
    C = 1.0
    max_iter = 200
    solver = 'liblinear'  # bueno para datasets pequeños y clasificación binaria
    mlflow.log_param("C", C)
    mlflow.log_param("max_iter", max_iter)
    mlflow.log_param("solver", solver)

    # Entrenar el modelo
    model = LogisticRegression(C=C, max_iter=max_iter, solver=solver)
    model.fit(X_train_scaled, y_train)

    # Predecir en el set de prueba
    y_pred = model.predict(X_test_scaled)

    # Calcular métricas
    acc = accuracy_score(y_test, y_pred)
    prec = precision_score(y_test, y_pred)
    rec = recall_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)

    # Registrar métricas en MLflow
    mlflow.log_metric("accuracy", acc)
    mlflow.log_metric("precision", prec)
    mlflow.log_metric("recall", rec)
    mlflow.log_metric("f1_score", f1)

    # Registrar tags
    mlflow.set_tags({
        "framework": "scikit-learn",
        "modelo": "Regresion Logistica",
        "proposito": "Clasificación binaria QSAR Biodegradation"
    })

    # Crear y guardar gráfico de matriz de confusión
    fig, ax = plt.subplots(figsize=(5,5))
    ConfusionMatrixDisplay.from_predictions(y_test, y_pred, ax=ax)
    plt.title("Matriz de Confusión")
    plt.savefig("confusion_matrix.png")
    plt.close(fig)

    # Registrar artefactos (gráfico)
    mlflow.log_artifact("confusion_matrix.png")

    # Registrar descripción en texto
    mlflow.log_text("Modelo entrenado con LogisticRegression de Scikit-learn sobre QSAR Biodegradation", "descripcion.txt")

    # Registrar modelo
    mlflow.sklearn.log_model(model, "modelo_logistico")

    print("✅ Run completado. Métricas registradas en MLflow")


2025/10/15 23:35:16 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/10/15 23:35:18 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


✅ Run completado. Métricas registradas en MLflow


## Paso 5

In [11]:
# 🔹 Importar librerías
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam
import mlflow
import mlflow.tensorflow

# 🔹 Activar autologging de TensorFlow/Keras
mlflow.tensorflow.autolog()

# 🔹 Definir experimento
mlflow.set_experiment("QSAR_Biodegradation_TensorFlowNN")

# 🔹 Crear el modelo secuencial
model = Sequential([
    Dense(64, activation='relu', input_shape=(X_train_scaled.shape[1],)),
    Dropout(0.2),
    Dense(32, activation='relu'),
    Dense(1, activation='sigmoid')  # salida binaria
])

# 🔹 Compilar modelo
model.compile(
    optimizer=Adam(learning_rate=0.001),
    loss='binary_crossentropy',
    metrics=['accuracy', tf.keras.metrics.Precision(), tf.keras.metrics.Recall()]
)

# 🔹 Entrenamiento dentro de MLflow
with mlflow.start_run(run_name="RedNeuronal_TF"):
    history = model.fit(
        X_train_scaled, y_train,
        validation_data=(X_test_scaled, y_test),
        epochs=20,
        batch_size=32,
        verbose=1
    )

    print("✅ Entrenamiento completado. Revisa la UI de MLflow para métricas y artefactos.")


C:\Users\Christian\Documents\mlflow 2\venv\Lib\site-packages\keras\src\layers\core\dense.py:92: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
2025/10/15 23:35:20 WARNING mlflow.tensorflow: Failed to log training dataset information to MLflow Tracking. Reason: 'Series' object has no attribute 'flatten'


Epoch 1/20
20/27 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.4947 - loss: 0.7238 - precision: 0.6511 - recall: 0.5417

27/27 ━━━━━━━━━━━━━━━━━━━━ 2s 22ms/step - accuracy: 0.6374 - loss: 0.6324 - precision: 0.7037 - recall: 0.7818 - val_accuracy: 0.7299 - val_loss: 0.5145 - val_precision: 0.7293 - val_recall: 0.9429
Epoch 2/20
20/27 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7852 - loss: 0.4861 - precision: 0.7855 - recall: 0.9351 

27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.8021 - loss: 0.4561 - precision: 0.8121 - recall: 0.9123 - val_accuracy: 0.8199 - val_loss: 0.4489 - val_precision: 0.8592 - val_recall: 0.8714
Epoch 3/20
21/27 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8055 - loss: 0.4230 - precision: 0.8595 - recall: 0.8461 

27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.8294 - loss: 0.3964 - precision: 0.8766 - recall: 0.8640 - val_accuracy: 0.8294 - val_loss: 0.4172 - val_precision: 0.8714 - val_recall: 0.8714
Epoch 4/20
22/27 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8494 - loss: 0.3482 - precision: 0.8681 - recall: 0.8969 

...

27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.8780 - loss: 0.2972 - precision: 0.9043 - recall: 0.9123 - val_accuracy: 0.8389 - val_loss: 0.3824 - val_precision: 0.8581 - val_recall: 0.9071
Epoch 9/20
24/27 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8853 - loss: 0.2817 - precision: 0.9156 - recall: 0.9136 

27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.8803 - loss: 0.2866 - precision: 0.9046 - recall: 0.9159 - val_accuracy: 0.8389 - val_loss: 0.3737 - val_precision: 0.8533 - val_recall: 0.9143
Epoch 10/20
25/27 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8852 - loss: 0.2991 - precision: 0.8911 - recall: 0.9367 

27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.8981 - loss: 0.2756 - precision: 0.9186 - recall: 0.9284 - val_accuracy: 0.8483 - val_loss: 0.3691 - val_precision: 0.8600 - val_recall: 0.9214
Epoch 11/20
27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.8886 - loss: 0.2797 - precision: 0.9029 - recall: 0.9320 - val_accuracy: 0.8436 - val_loss: 0.3781 - val_precision: 0.8591 - val_recall: 0.9143
Epoch 12/20
27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.8803 - loss: 0.2734 - precision: 0.9046 - recall: 0.9159 - val_accuracy: 0.8483 - val_loss: 0.3748 - val_precision: 0.8553 - val_recall: 0.9286
Epoch 13/20
27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.8922 - loss: 0.2682 - precision: 0.9091 - recall: 0.9302 - val_accuracy: 0.8483 - val_loss: 0.3750 - val_precision: 0.8600 - val_recall: 0.9214
Epoch 14/20
27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.9028 - loss: 0.2573 - precision: 0.9206 - recall: 0.9338 - val_accuracy: 0.8436 - val_loss: 0.3770 - val_preci

2025/10/15 23:35:29 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.


✅ Entrenamiento completado. Revisa la UI de MLflow para métricas y artefactos.


## Paso 6

In [12]:
import mlflow

# Seleccionar el experimento por nombre
experiment = mlflow.get_experiment_by_name("QSAR_Biodegradation_LogisticRegression")
experiment_id = experiment.experiment_id

# Buscar todos los runs
runs = mlflow.search_runs(
    experiment_ids=[experiment_id],
    filter_string="",  # Puedes filtrar por tags o nombre: e.g. "tags.modelo = 'Regresion Logistica'"
    order_by=["metrics.accuracy DESC"]  # Ordenar por accuracy descendente
)

# Mostrar las primeras filas con métricas y parámetros
print(runs[['run_id', 'metrics.accuracy', 'metrics.precision', 'metrics.recall', 'metrics.f1_score', 'params.C', 'params.max_iter', 'params.solver']])


                             run_id  metrics.accuracy  metrics.precision  \
0  70a62eb889f949f3a8f1328d2c922e41          0.853081           0.881119   
1  50047884af344c29a957b209258f84a6          0.853081           0.881119   
2  c42de85b144e436bbd8d86afb3fb11a5          0.853081           0.881119   
3  238e73c2f7a44764be5b4f833a1c85cc          0.853081           0.881119   
4  f1cadd21cd64416da2beef109ec381f3               NaN                NaN   
5  228a1bf0db754bb982f4fb9f0fb1da88               NaN                NaN   
6  35f6efabb92d42328133ae719fca61b3               NaN                NaN   
7  4d110f7ab22a4398ae96bad0d99c38c4               NaN                NaN   

   metrics.recall  metrics.f1_score params.C params.max_iter params.solver  
0             0.9          0.890459      1.0             200     liblinear  
1             0.9          0.890459      1.0             200     liblinear  
2             0.9          0.890459      1.0             200     liblinear  
3      

In [13]:
runs.to_csv("comparacion_runs.csv", index=False)

## Paso 7

In [16]:
from openai import OpenAI
import mlflow
import os

# Configurar el cliente Ollama local
client = OpenAI(base_url="http://localhost:11434/v1", api_key="ollama")

# Modelo que descargaste
modelo = "llama3"

# Lista de preguntas sobre los modelos
preguntas = [
    "¿Qué significa obtener un F1-score de 0.88?",
    "¿Por qué una red neuronal podría tener mejor recall que una regresión logística?",
    "¿Qué podría mejorar el rendimiento de los modelos en este dataset?"
]

respuestas = []

for p in preguntas:
    response = client.chat.completions.create(
        model=modelo,
        messages=[{"role": "user", "content": p}]
    )
    r = response.choices[0].message.content.strip()
    respuestas.append(f"Pregunta: {p}\nRespuesta: {r}\n")

# Guardar en archivo
archivo = "interpretacion_ollama.txt"
with open(archivo, "w", encoding="utf-8") as f:
    f.writelines(respuestas)

# Registrar en MLflow
mlflow.set_experiment("QSAR_Biodegradation_Interpretacion")
with mlflow.start_run(run_name="Interpretacion_Ollama_Notebook"):
    mlflow.log_artifact(archivo)
    print("✅ Respuestas de Ollama registradas en MLflow")


2025/10/15 23:40:17 INFO mlflow.tracking.fluent: Experiment with name 'QSAR_Biodegradation_Interpretacion' does not exist. Creating a new experiment.


✅ Respuestas de Ollama registradas en MLflow


## Paso 8

In [17]:
import mlflow
import mlflow.sklearn
import mlflow.tensorflow

# Activar autologging global
mlflow.autolog()
mlflow.sklearn.autolog()
mlflow.tensorflow.autolog()


2025/10/15 23:41:07 INFO mlflow.tracking.fluent: Autologging successfully enabled for keras.
2025/10/15 23:41:08 INFO mlflow.tracking.fluent: Autologging successfully enabled for openai.
2025/10/15 23:41:09 INFO mlflow.tracking.fluent: Autologging successfully enabled for sklearn.


In [19]:
from sklearn.model_selection import KFold
from sklearn.linear_model import LogisticRegression
import numpy as np

kf = KFold(n_splits=3, shuffle=True, random_state=42)

with mlflow.start_run(run_name="LogisticRegression_CV") as parent_run:
    for fold, (train_idx, val_idx) in enumerate(kf.split(X_train_scaled)):
        X_train_fold, X_val_fold = X_train_scaled[train_idx], X_train_scaled[val_idx]
        y_train_fold, y_val_fold = y_train.iloc[train_idx], y_train.iloc[val_idx]  # <- CORRECTO

        with mlflow.start_run(run_name=f"Fold_{fold+1}", nested=True):
            model = LogisticRegression(C=1.0, max_iter=200, solver='liblinear')
            model.fit(X_train_fold, y_train_fold)
            acc = model.score(X_val_fold, y_val_fold)
            mlflow.log_metric("accuracy", acc)
            print(f"Fold {fold+1} Accuracy: {acc:.4f}")


2025/10/15 23:42:24 WARNING mlflow.sklearn: Failed to log training dataset information to MLflow Tracking. Reason: 'Series' object has no attribute 'flatten'
2025/10/15 23:42:27 WARNING mlflow.sklearn: Failed to log training dataset information to MLflow Tracking. Reason: 'Series' object has no attribute 'flatten'


Fold 1 Accuracy: 0.8759


2025/10/15 23:42:30 WARNING mlflow.sklearn: Failed to log training dataset information to MLflow Tracking. Reason: 'Series' object has no attribute 'flatten'


Fold 2 Accuracy: 0.8577
Fold 3 Accuracy: 0.8683
